<a href="https://colab.research.google.com/github/isys5002-itp/isys5002-2023-semester2/blob/main/09_sqlite3_notebook_Thur4PM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#SQL and Python (and a little Pandas)

When it comes to data retrieval and basic statistics, SQL shines, while Python shines for in-depth, flexible exploratory data analysis or data science. What if you could combine the two programming languages into a single application?

We will use SQLite3 which is part of the standard Python 3 package, so nothing to install. Let’s take a quick look at the data types that are available:

| SQL | Python |
|-----|--------|
| NULL — Includes a NULL value | none
| INTEGER — Includes an integer | int
| REAL — Includes a floating-point (decimal) value | float
| TEXT. — Includes text | str
| BLOB. — Includes a binary large object that is stored exactly as input | bytes

In this notebook, we’ll demonstrate

* Loading the library
* Creating and connecting to your database
* Creating database tables
* Adding data
* Querying data
* Deleting data



## Import the library

Let’s start off the tutorial by loading in the library. We can do this by using the following command

In [1]:
import sqlite3

## Create a connection

We use the connect method and pass the name of the database. Let's create an *superheroes.db*.

In [2]:
conn = sqlite3.connect('superheroes.db')

We’ve created a new connection object, as well as a new file called *superheroes.db* in the directory in which you’re working.


## Create a cursor object

Now that we’ve created a database connection object, our next task is to create a cursor object. A cursor object allows us to execute SQL queries against a database. A cursor acts a middleware between a connection and SQL query

In [3]:
cur = conn.cursor() #something like 'row' returned from a table

# Create Table

Use a common 'pattern'

1. Create a SQL command as a string
2. User cursor to execture the command
3. If needed fetch the results

In [4]:
sql = """
CREATE TABLE IF NOT EXISTS users(
   userid INT PRIMARY KEY,
   fname TEXT,
   lname TEXT,
   superhero TEXT,
   gender TEXT);
"""

# Execute query and commit transaction

cur.execute(sql)
conn.commit()

The IF NOT EXISTS will help us when reconnecting to the database. The query will allow us to check if the table exists, and if it does, nothing is changed.

# Insert into Table

Let’s take a look at how to add data with SQLite in Python to the database we just created. Similar to the table generation query, the query to add data uses the cursor object to execute the query.

In [5]:
'''
INSERT INTO users(userid, fname, lname, superhero, gender)
   VALUES('00001', 'Bruce', 'Wayne', 'Batman', 'male');
'''

"\nINSERT INTO users(userid, fname, lname, superhero, gender)\n   VALUES('00001', 'Bruce', 'Wayne', 'Batman', 'male');\n"

In [6]:
sql = '''
INSERT INTO users(userid, fname, lname, superhero, gender)
   VALUES('00006', 'Bruce', 'Wayne', 'Batman', 'male');
'''

# Execute query and commit transaction
cur.execute(sql)
conn.commit()

In [7]:
sql = '''
SELECT * FROM users;
'''
cur.execute(sql)
all_results = cur.fetchall()
(all_results)

[(6, 'Bruce', 'Wayne', 'Batman', 'male')]

Often, when we’re working within Python, we’ll have variables that hold values for us. For example, we may have a tuple that contains that information about a user which might look like this:

In [8]:
user = ('00002', 'Sue', 'Storm', 'Invisible Woman', 'female') # MUST BE A TUPLE!

If we wanted to load this data into our database, we would use a different convention:



In [9]:
cur.execute("INSERT INTO users VALUES(?, ?, ?, ?, ?);", user)
conn.commit()

In [10]:
sql = '''
SELECT * FROM users;
'''
cur.execute(sql)
all_results = cur.fetchall()
(all_results)

[(6, 'Bruce', 'Wayne', 'Batman', 'male'),
 (2, 'Sue', 'Storm', 'Invisible Woman', 'female')]

Incidentally, using the (?, ?, …) method we noted above also helps protect against SQL injection attacks.

It’s important to note here that the **SQLite expects the values to be in tuple-format**. However, the variable can contain a list, as long as the list items are tuples. For example, we could add more users using the variable:

In [11]:
more_users = [('00003', 'Peter', 'Parker', 'Spider Man', 'male'),
              ('00004', 'Tony', 'Stark', 'Ironman', 'male'),
              ('00005', 'Diana', 'Princess', 'Wonder Woman', 'female')]

In [12]:
cur.executemany("INSERT INTO users VALUES(?, ?, ?, ?, ?);", more_users)
conn.commit()

In this case, instead of using the execute function, we’ll want to use the executemany function

## Fetching Data

Here we take a look at how to select data with SQLite in Python! We’ll follow a similar structure as we did to execute queries above

In [13]:
sql = '''
SELECT * FROM users;
'''
rows = cur.execute(sql)
for row in rows:
  print(row)

(6, 'Bruce', 'Wayne', 'Batman', 'male')
(2, 'Sue', 'Storm', 'Invisible Woman', 'female')
(3, 'Peter', 'Parker', 'Spider Man', 'male')
(4, 'Tony', 'Stark', 'Ironman', 'male')
(5, 'Diana', 'Princess', 'Wonder Woman', 'female')


Some useful methods

In [14]:
sql = '''
SELECT * FROM users;
'''
cur.execute(sql)
one_result = cur.fetchone()
print(one_result)

(6, 'Bruce', 'Wayne', 'Batman', 'male')


We wanted to return more than only one result, we could use the fetchmany()

In [15]:
sql = '''
SELECT * FROM users;
'''
cur.execute(sql)
three_results = cur.fetchmany(3)
print(three_results)

[(6, 'Bruce', 'Wayne', 'Batman', 'male'), (2, 'Sue', 'Storm', 'Invisible Woman', 'female'), (3, 'Peter', 'Parker', 'Spider Man', 'male')]


We canuse the fetchall() function to return all the results.

In [19]:
sql = '''
SELECT * FROM users;
'''
cur.execute(sql)
all_results = cur.fetchall()
(all_results)

[(6, 'Bruce', 'Wayne', 'Batman', 'male'),
 (2, 'Sue', 'Storm', 'Invisible Woman', 'female'),
 (3, 'Peter', 'Parker', 'Spider Man', 'male'),
 (4, 'Tony', 'Stark', 'Ironman', 'male'),
 (5, 'Diana', 'Princess', 'Wonder Woman', 'female')]

In [21]:
sql = '''
SELECT superhero, gender FROM users;
'''
cur.execute(sql)
all_results = cur.fetchall()
(all_results)

[('Batman', 'male'),
 ('Invisible Woman', 'female'),
 ('Spider Man', 'male'),
 ('Ironman', 'male'),
 ('Wonder Woman', 'female')]

In [22]:
# extract last name and gender of the list of superheroes, order lname, gender, sp

sql = '''
SELECT lname, gender, superhero FROM users;
'''
cur.execute(sql)
all_results = cur.fetchall()
(all_results)



[('Wayne', 'male', 'Batman'),
 ('Storm', 'female', 'Invisible Woman'),
 ('Parker', 'male', 'Spider Man'),
 ('Stark', 'male', 'Ironman'),
 ('Princess', 'female', 'Wonder Woman')]

In [25]:
# extract records of female superheroes from the 'users' table
sql = '''
SELECT fname, lname, superhero
FROM users
WHERE gender == 'female';
'''
cur.execute(sql)
all_results = cur.fetchall()
(all_results)


[('Sue', 'Storm', 'Invisible Woman'), ('Diana', 'Princess', 'Wonder Woman')]

## Delete Data

In [26]:
sql='''
DELETE FROM users WHERE userid='6';
'''
cur.execute(sql)
conn.commit()

In [28]:
sql='''
DELETE FROM users WHERE lname='Parker';
'''
cur.execute(sql)
conn.commit()

In [29]:
sql = '''
SELECT * FROM users;
'''
cur.execute(sql)
all_results = cur.fetchall()
(all_results)

[(2, 'Sue', 'Storm', 'Invisible Woman', 'female'),
 (4, 'Tony', 'Stark', 'Ironman', 'male'),
 (5, 'Diana', 'Princess', 'Wonder Woman', 'female')]

Did it work?

In [30]:
sql='''
select * from users where lname='Parker';
'''
cur.execute(sql)
cur.fetchall()

[]

This prints out an empty list, confirming that the record has been deleted.

## pandas

We can use the pandas package to read a SQLite database

In [36]:
import sqlite3
connection = sqlite3.connect('superheroes_all.db')
cursor = conn.cursor()

In [37]:
# view schema
sql='''
SELECT name FROM sqlite_master WHERE type='table';
'''
cursor.execute(sql)
cursor.fetchall()

[('your_table',), ('superheroes',)]

In [ ]:
import pandas as pd

sql='''
SELECT * from superheroes;
'''
# Load the data into a DataFrame
users_df = pd.read_sql_query(sql, connection)
users_df

and create table from a dataframe

In [42]:
# Select only data for females
users_female_df = users_df[users_df.gender == 'Female']

users_female_df

,id,last_name,first_name,superhero,gender,alignment
1,2,Prince,Diana,Wonder Woman,Female,Good
4,5,Maximoff,Wanda,Scarlet Witch,Female,Good
13,14,Quinzel,Harley,Harley Quinn,Female,Evil
14,15,Kyle,Selina,Catwoman,Female,Evil
15,16,Darkholme,Mystique,Mystique,Female,Evil
17,18,Ivy,Poison,Poison Ivy,Female,Evil
18,19,Moone,Enchantress,Enchantress,Female,Evil
19,20,Romanoff,Natasha,Black Widow,Female,Good
20,21,Danvers,Carol,Captain Marvel,Female,Good


In [43]:
# Write the new DataFrame to a new SQLite table
users_female_df.to_sql("females", conn, if_exists='replace')

9

In [44]:
sql='''
SELECT name FROM sqlite_master WHERE type='table';
'''
cur.execute(sql)
cur.fetchall()

[('your_table',), ('superheroes',), ('females',)]

## Activity

- insert Spiderman info
- create 'male' sperheroes table

In [45]:
import sqlite3
conn = sqlite3.connect('superheroes.db')
cur = conn.cursor()

In [ ]:

sql = '''
INSERT INTO users(userid, fname, lname, superhero, gender)
   VALUES('00003', 'Peter', 'Parker', 'Spiderman', 'male');
'''

# Execute query and commit transaction
cur.execute(sql)
conn.commit()

In [50]:
import pandas as pd

sql='''
SELECT * from users;
'''
# Load the data into a DataFrame
users_df = pd.read_sql_query(sql, conn)
users_df

,userid,fname,lname,superhero,gender
0,2,Sue,Storm,Invisible Woman,female
1,4,Tony,Stark,Ironman,male
2,5,Diana,Princess,Wonder Woman,female
3,3,Peter,Parker,Spiderman,male


In [51]:
# Select only data for males

users_male_df = users_df[users_df.gender == 'male']

users_male_df




,userid,fname,lname,superhero,gender
1,4,Tony,Stark,Ironman,male
3,3,Peter,Parker,Spiderman,male


In [52]:
# Write the new DataFrame to a new SQLite table
users_male_df.to_sql("males", conn, if_exists='replace')

2

## View Schema

Was the new table added.  Lets have a look at the schema.

In [53]:
sql='''
SELECT name FROM sqlite_master WHERE type='table';
'''
cur.execute(sql)
cur.fetchall()

[('users',), ('males',)]

Now lets get details on a table

In [54]:
sql='''
PRAGMA table_info('users');
'''
cur.execute(sql)
cur.fetchall()

[(0, 'userid', 'INT', 0, None, 1),
 (1, 'fname', 'TEXT', 0, None, 0),
 (2, 'lname', 'TEXT', 0, None, 0),
 (3, 'superhero', 'TEXT', 0, None, 0),
 (4, 'gender', 'TEXT', 0, None, 0)]

## Delete (Drop) Table

When the IF EXISTS clause is used along with DROP TABLE syntax SQLite will not report any error message if the table does not exist.

In [55]:
sql='''
DROP TABLE IF EXISTS males;
'''
cur.execute(sql)
cur.fetchall()

[]

Did it work?

In [56]:
sql='''
SELECT name FROM sqlite_master WHERE type='table';
'''
cur.execute(sql)
cur.fetchall()

[('users',)]

In [57]:
sql = '''
SELECT * FROM users;
'''
cur.execute(sql)
all_results = cur.fetchall()
(all_results)

[(2, 'Sue', 'Storm', 'Invisible Woman', 'female'),
 (4, 'Tony', 'Stark', 'Ironman', 'male'),
 (5, 'Diana', 'Princess', 'Wonder Woman', 'female'),
 (3, 'Peter', 'Parker', 'Spiderman', 'male')]

## Activity - Order the table by 'userid' and display the result

In [59]:
sql = '''
SELECT * FROM users ORDER BY userid DESC
'''
cur.execute(sql)
all_results = cur.fetchall()
(all_results)

[(5, 'Diana', 'Princess', 'Wonder Woman', 'female'),
 (4, 'Tony', 'Stark', 'Ironman', 'male'),
 (3, 'Peter', 'Parker', 'Spiderman', 'male'),
 (2, 'Sue', 'Storm', 'Invisible Woman', 'female')]

## Close the connection

The connection must be closed at the end of the session

In [60]:
conn.close()